# Multiclass Classification to detect stress source type
Classes:
- verticillium
- cycloconium, 
- unidentified_stress 
- healthy

Steps we need to follow:
- set a threshold to determine if a sample will be considered stressed (based on previous analysis we select 5%)
- check which factor provides the highest stress percentage (eg. Verticillium = 5 and is the greatest value between the three recorded sources) and label the sample as Vert
- apply multiclass classification on the created columns as created

## STEP 1: Load Libraries and the initial datasets


In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import sklearn.model_selection as skl
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
# from sklearn.ensemble import RandomForestClassifier as rf
from sklearn import metrics as skm
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder

In [ ]:
sampling_data = pd.read_csv('data/olives_s2_sampling_data_2019-2020.csv')
sampling_data

In [ ]:
training_features = pd.read_csv('data/ML_sentinel2_training_data_19_20.csv')
training_features

In [ ]:
training_features.columns

In [ ]:
sampling_data.describe()

In [ ]:
# These lists will be populated and will become the values for the dictionary's keys
sulabels = []
strtypelabels = []

In [ ]:
# Create a dictionary that will be use to create a dataframe with the new classes in a column
labels_dic = {'su':sulabels,'stress_type':strtypelabels}

In [ ]:
for i in range(len(sampling_data)):
    if sampling_data.total_stress[i] < 0.06:
        labels_dic['su'].append(sampling_data.sampling_unit_codes[i])
        labels_dic['stress_type'].append('healthy')
    else:
        if sampling_data.verticillium[i] >= sampling_data.cycloconium[i] and sampling_data.verticillium[i] > sampling_data.usf[i]:
            labels_dic['su'].append(sampling_data.sampling_unit_codes[i])
            labels_dic['stress_type'].append('verticillium')
        if sampling_data.cycloconium[i] > sampling_data.verticillium[i] and sampling_data.cycloconium[i] > sampling_data.usf[i]:
            labels_dic['su'].append(sampling_data.sampling_unit_codes[i])
            labels_dic['stress_type'].append('spilocaea')
        if sampling_data.usf[i] >= sampling_data.verticillium[i] and sampling_data.usf[i] >= sampling_data.cycloconium[i]:
            labels_dic['su'].append(sampling_data.sampling_unit_codes[i])
            labels_dic['stress_type'].append('unidentified')

In [ ]:
# We will use the dictionary, created in the previous step, to create a new dataframe
labels = pd.DataFrame(labels_dic, columns=['su','stress_type'])
labels

In [ ]:
# The following is a simple way to add new columns to an existing dataframe
sampling_data['stress_type'] = labels['stress_type']
sampling_data

In [ ]:
sampling_data['stress_type'].describe()

## STEP 2: Merge the data frame containing the labeled data with the one containing the training features

In [ ]:
columns_to_add = ['NDVI', 'RDVI', 'GNDVI', 'SAVI', 'OSAVI', 'EVI', 'TVI', 'MTVI1',
       'TCARI', 'MCARI', 'MSR', 'CARI', 'PSSRb', 'PSSRc', 'TCARI/OSAVI',
       'Redness', 'ARVI', 'ARI1', 'ARI2', 'CRI1', 'CRI2', 'B02', 'B03', 'B04', 'B08']

In [ ]:
sampling_data[columns_to_add] = training_features[columns_to_add]
sampling_data

## STEP 3: Remove the outliers from the dataset

In [ ]:
q1 = sampling_data["total_stress"].quantile(0.25)
q3 = sampling_data["total_stress"].quantile(0.75)

iqr = q3 - q1
cut_off = iqr * 1.5

lower_threshold = q1 - cut_off
upper_threshold = q3 + cut_off

sampling_data_clrd = sampling_data[(sampling_data.total_stress > lower_threshold) & (sampling_data.total_stress < upper_threshold)]

In [ ]:
# reset the indices after removing outliers and check the dataset

sampling_data_clrd.reset_index(drop=True, inplace=True)

sampling_data_clrd

In [ ]:
sampling_data_clrd.describe()

## STEP 4.1: Create dummy variables for string columns in **features**

## STEP 4.2: Drop unnecessary columns

In [ ]:
sampling_data_clrd.columns

In [ ]:
to_drop = ['sampling_unit_codes',
           'verticillium',
           'cycloconium',
           'usf',
           'total_stress',
           'su_surface',
           'su_tree_surface',
           'su_background_surface',
           'su_veg_percent',
           'su_backgr_percent'
          ]

data = sampling_data_clrd.drop(to_drop, axis=1)
data
# A different way below, but it raises an issue
# sampling_data_clrd.drop(to_drop, axis=1, inplace=True)

## STEP 5: Use an oversampling technique to equalize the class samples (SMOTE)

In [ ]:
# y = data.stress_type

In [ ]:
# check the ditribution of the classes
data.stress_type.value_counts()

In [ ]:
# X would be the following
data[data.columns[1:]]

In [ ]:
# and y the following:
data.stress_type.to_excel('results\pathogen_classes.xlsx')

In [ ]:
sns.set_theme(style="darkgrid")

In [ ]:
X = data[data.columns[1:]]
y = data.stress_type[:]
# label encode the target variable
y_encoded = LabelEncoder().fit_transform(y)
# summarize distribution
counter = Counter(y_encoded)
for k,v in counter.items():
	per = v / len(y_encoded) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

**By default, SMOTE will oversample all classes to have the same number of examples as the class with the most examples.**

In [ ]:
# X = data[data.columns[1:]]
# y = data.stress_type[:]

# # label encode the target variable
# y = LabelEncoder().fit_transform(y)
# # transform the dataset
# oversample = SMOTE(random_state=0)
# X, y = oversample.fit_resample(X, y)
# # summarize distribution
# counter = Counter(y)
# for k,v in counter.items():
# 	per = v / len(y) * 100
# 	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# # plot the distribution
# plt.bar(counter.keys(), counter.values())
# plt.show()

## STEP 6: APPLY CLASSIFICATION ALGORITHM

In [ ]:
# LabelEncoder().inverse_transform(y)

In [ ]:
# OneVsRestClassifier?

### The classifier that provided the best accuracy for general stress detection was quadratic_discriminant when applied to infection threshold 6% on vegetation surface to classify as stressed, with the parameters given below

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda

In [ ]:
# OneVsRestClassifier?

# load feature and label data on variables
X, y = data[data.columns[1:]], data.stress_type[:]

# transform the label categories (healthy, vert, cyclo, usf) to numerical class digits (class 1, class 2 etc.)
# y = LabelEncoder().fit_transform(y)

# define and apply the oversampling technique to the entire dataset
oversample = SMOTE(random_state=0)
X_sm, y_sm = oversample.fit_resample(X, y)                       

In [ ]:
y_sm_enc = LabelEncoder().fit_transform(y_sm)
# summarize distribution after oversampling with SMOTE
counter = Counter(y_sm_enc)
for k,v in counter.items():
	per = v / len(y_sm_enc) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

In [ ]:
# divide the data in training and test sets
X_train, X_test, y_train, y_test = skl.train_test_split(X_sm, y_sm, test_size=0.25, random_state=0)

# define the classifier to be used as clf
clf = OneVsRestClassifier(qda(
    reg_param=0.0,
    store_covariance=True,
    tol=0.001
    ))

In [ ]:
# fit the training data in the classifier and get the predicted values by applying the classifier to the test set
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

In [ ]:
# clf.score()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
plt.rcParams.update({'font.size': 14})

ax.set_title('OvR Confusion Matrix\n', fontsize=20)
ax.set_xlabel('\nPredicted label', fontsize=16)
ax.set_ylabel('True label', fontsize=16)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(14)


plt.grid(False)
skm.plot_confusion_matrix(clf, X_test, y_test, cmap='bone',ax=ax)
# skm.confusion_matrix(y_test, y_pred)

In [ ]:
fig.savefig('D:\Dropbox\Publications\sentinel-2 olive tree stress detection\Results & Discussion\\figures\conf_matrix_20221001.jpg')

In [ ]:
multiclass_cm = skm.confusion_matrix(y_test, y_pred)

multiclass_cm_df = pd.DataFrame(multiclass_cm)

In [ ]:
plt.figure(figsize=(8,7))
hm = sns.heatmap(multiclass_cm_df, annot=True,cmap="Greys")
hm.xaxis.set_ticklabels(hm.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=14)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

## STEP 7: ACQUIRE VISUALIZATIONS AND METRICS

### 7.1 Create Functions to calculate tpr, fpr, roc coords and plot the roc curve

In [ ]:
def calculate_tpr_fpr(y_real, y_pred):
    '''
    Calculates the True Positive Rate (tpr) and the True Negative Rate (fpr) based on real and predicted observations
    
    Args:
        y_real: The list or series with the real classes
        y_pred: The list or series with the predicted classes
        
    Returns:
        tpr: The True Positive Rate of the classifier
        fpr: The False Positive Rate of the classifier
    '''
    
    # Calculates the confusion matrix and recover each element
    cm = confusion_matrix(y_real, y_pred)
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]
    
    # Calculates tpr and fpr
    tpr =  TP/(TP + FN) # sensitivity - true positive rate
    fpr = 1 - TN/(TN+FP) # 1-specificity - false positive rate
    
    return tpr, fpr

In [ ]:
def get_all_roc_coordinates(y_real, y_proba):
    '''
    Calculates all the ROC Curve coordinates (tpr and fpr) by considering each point as a treshold for the predicion of the class.
    
    Args:
        y_real: The list or series with the real classes.
        y_proba: The array with the probabilities for each class, obtained by using the `.predict_proba()` method.
        
    Returns:
        tpr_list: The list of TPRs representing each threshold.
        fpr_list: The list of FPRs representing each threshold.
    '''
    tpr_list = [0]
    fpr_list = [0]
    for i in range(len(y_proba)):
        threshold = y_proba[i]
        y_pred = y_proba >= threshold
        tpr, fpr = calculate_tpr_fpr(y_real, y_pred)
        tpr_list.append(tpr)
        fpr_list.append(fpr)
    return tpr_list, fpr_list

In [ ]:
def plot_roc_curve(tpr, fpr, scatter = True, ax = None):
    '''
    Plots the ROC Curve by using the list of coordinates (tpr and fpr).
    
    Args:
        tpr: The list of TPRs representing each coordinate.
        fpr: The list of FPRs representing each coordinate.
        scatter: When True, the points used on the calculation will be plotted with the line (default = True).
    '''
    if ax == None:
        plt.figure(figsize = (5, 5))
        ax = plt.axes()
    
    if scatter:
        sns.scatterplot(x = fpr, y = tpr, ax = ax)
    sns.lineplot(x = fpr, y = tpr, ax = ax)
    sns.lineplot(x = [0, 1], y = [0, 1], color = 'green', ax = ax)
    plt.xlim(-0.05, 1.05)
    plt.ylim(-0.05, 1.05)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")

In [ ]:
# Plots the Probability Distributions and the ROC Curves One vs Rest
fig2 = plt.figure(figsize = (12, 8))
bins = [i/20 for i in range(20)] + [1]
classes = clf.classes_
roc_auc_ovr = {}
for i in range(len(classes)):
    # Gets the class
    c = classes[i]
    
    # Prepares an auxiliar dataframe to help with the plots
    df_aux = X_test.copy()
    df_aux['class'] = [1 if y == c else 0 for y in y_test]
    df_aux['prob'] = y_proba[:, i]
    df_aux = df_aux.reset_index(drop = True)
    
    # Plots the probability distribution for the class and the rest
    ax = plt.subplot(2, 4, i+1) # 2 is for the number of rows, 4 for the number of classes and i+1 to start plotting a specific plot starting from 1 for the top left
    sns.histplot(x = "prob", data = df_aux, hue = 'class', color = 'b', ax = ax, bins = bins)
    ax.set_title(c)
    ax.legend([f"Class: {c}", "Rest"])
    ax.set_xlabel(f"P(x = {c})")
    
    # Calculates the ROC Coordinates and plots the ROC Curves
    ax_bottom = plt.subplot(2, 4, i+5) # i+5 is to skip the first four plots, and apply the subplot on the fifth
    tpr, fpr = get_all_roc_coordinates(df_aux['class'], df_aux['prob'])
    plot_roc_curve(tpr, fpr, scatter = False, ax = ax_bottom)
    ax_bottom.set_title("ROC Curve OvR")
    
    # Calculates the ROC AUC OvR
    roc_auc_ovr[c] = skm.roc_auc_score(df_aux['class'], df_aux['prob'])
plt.tight_layout()

In [ ]:
fig2.savefig('D:\Dropbox\Publications\sentinel-2 olive tree stress detection\Results & Discussion\\figures\ovr_class_comparison_20221001.jpg')

### 7.2 Acquire metrics for the classes

In [ ]:
# Displays the ROC AUC for each class
avg_roc_auc = 0
i = 0
for k in roc_auc_ovr:
    avg_roc_auc += roc_auc_ovr[k]
    i += 1
    print(f"{k} ROC AUC OvR: {roc_auc_ovr[k]:.4f}")
print(f"average ROC AUC OvR: {avg_roc_auc/i:.4f}")

In [ ]:
accuracy = skm.accuracy_score(y_test,y_pred)
accuracy

In [ ]:
print(skm.classification_report(y_test, y_pred))

## Descriptive Statistics

In [ ]:
sns.set_theme(style="darkgrid")
sns.catplot(x="stress_type", y="total_stress", data=sampling_data)

In [ ]:
sns.displot(sampling_data.total_stress, bins=10)

In [ ]:
plt.subplots(figsize=(12,8))
# plt.grid(False)
sns.scatterplot(data=sampling_data_clrd, y="su_veg_percent", x="total_stress", hue="stress_type", s=100)

In [ ]:
plt.subplots(figsize=(16,8))

# sns.lineplot(data=sampling_data, x="su_veg_percent", y="total_stress")
sns.lineplot(data=sampling_data_clrd, x="su_veg_percent", y="NDVI", hue="stress_type")
# sns.lineplot(data=sampling_data, x="su_backgr_percent", y="total_stress")


In [ ]:
sampling_data_clrd.columns